## sequence is updating the hidden layer

**State** is (1) gradient and (2) latent space of all layers up to not including output layer, and (3) task identifying information.

**Action** is nxn dimension nudge to layers from -0.01 to 0.01 or -0.001 to 0.001

**Reward** is a difference between developed layer and oracle layer (trained during setup)

Continual learning where it keeps nudging the layer closer to oracle.

Imp'l details
- train oracle layer in initialization
- each step, update target layer in self.layers with nudge
- get difference between target layer being developed and the oracle layer
- use this differnece as the reward

Less complex than discrete
- no need for layer pool
- no need for task enum
- no need for error conditions and complex action logic

In [36]:
import math
import copy 
import random
import datetime
from collections import defaultdict
import json
import numpy as np 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import gymnasium
from typing import (
    Type,
    List,
    Tuple,
)
from sb3_contrib import RecurrentPPO
from stable_baselines3 import PPO, A2C, SAC, TD3
import wandb
import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science','ieee', 'notebook', 'bright'])
plt.rcParams.update({'figure.dpi': '75'})

## config

In [57]:
default_config = {
    'seed' : 41,
    'device' : 'cuda',
    'n_runs' : 1,
    'epochs' : 3,
    'timesteps' : 1000,
    'n_x' : 100,
    'n_tasks' : 2,
    'task_min_loss' : defaultdict(lambda: None),
    'task_max_loss' : defaultdict(lambda: None),
    'in_features' : 1,
    'out_features' : 1,
    'n_hidden_layers_per_network' : 1,
    'n_layers_per_network' : 3,
    'n_nodes_per_layer' : 4,
    'pool_layer_type' : torch.nn.Linear,
    'batch_size' : 100,
    'learning_rate' : 0.01,
    'action_cache_size' : 5,
    'num_workers' : 0,
    'loss_fn' : torch.nn.MSELoss(),
    'sb3_model' : 'PPO',
    'sb3_policy' : 'MlpPolicy',
    'log_dir' : 'wandb',
    }
config = default_config
config['n_pool_hidden_layers'] = config['n_tasks'] * config['n_hidden_layers_per_network']
config

{'seed': 41,
 'device': 'cuda',
 'n_runs': 1,
 'epochs': 3,
 'timesteps': 1000,
 'n_x': 100,
 'n_tasks': 2,
 'task_min_loss': defaultdict(<function __main__.<lambda>()>, {}),
 'task_max_loss': defaultdict(<function __main__.<lambda>()>, {}),
 'in_features': 1,
 'out_features': 1,
 'n_hidden_layers_per_network': 1,
 'n_layers_per_network': 3,
 'n_nodes_per_layer': 4,
 'pool_layer_type': torch.nn.modules.linear.Linear,
 'batch_size': 100,
 'learning_rate': 0.01,
 'action_cache_size': 5,
 'num_workers': 0,
 'loss_fn': MSELoss(),
 'sb3_model': 'PPO',
 'sb3_policy': 'MlpPolicy',
 'log_dir': 'wandb',
 'n_pool_hidden_layers': 2}

In [58]:
wandb.init(
    project='reinforcement-meta-learning',
    config=config
)

cumulative_loss_task0_per_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃█▁▁
cumulative_reward_task0_per_epoch,████████████████▆▁██
loss_task0_per_step,▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_task1_per_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▁▁▁▁▂▂▄▅▆▇█
reward_task0_per_step,█████████▁████████████████████████████▂█
reward_task1_per_step,████████████████████████▇▇▆▇▇██▇█▇▇▅▄▃▂▁
cumulative_loss_task0_per_epoch,6704.28662
cumulative_reward_task0_per_epoch,-30707.44531
loss_task0_per_step,0.03784
loss_task1_per_step,2997.6228
reward_task0_per_step,-100.11072


In [59]:
torch.cuda.is_available()

True

### regression
sinusoidal curve regression as in MAML 2018 paper

In [60]:
lower_bound = torch.tensor(-5).float()
upper_bound = torch.tensor(5).float()
X = np.linspace(lower_bound, upper_bound, config['n_x'])
amplitude_range = torch.tensor([0.1, 5.0]).float()
phase_range = torch.tensor([0, math.pi]).float()
amps = torch.from_numpy(np.linspace(amplitude_range[0], amplitude_range[1], config['n_tasks'])).float()
phases = torch.from_numpy(np.linspace(phase_range[0], phase_range[1], config['n_tasks'])).float()
tasks_data = torch.tensor(np.array([ 
        X
        for _ in range(config['n_tasks'])
        ])).float()
tasks_targets = torch.tensor(np.array([
        [((a * np.sin(x)) + p).float()
        for x in X] 
        for a, p in zip(amps, phases)
        ])).float()
tasks_info = [
        {'i' : i, 
         'amp' : a, 
         'phase_shift' : p, 
         'lower_bound' : lower_bound, 
         'upper_bound' : upper_bound, 
         'amplitude_range_lower_bound' : amplitude_range[0], 
         'amplitude_range_upper_bound' : amplitude_range[1], 
         'phase_range_lower_bound' : phase_range[0],
         'phase_range_lower_bound' : phase_range[1]}
        for i, (a, p) in enumerate(zip(amps, phases))
]
print(f'[INFO] Tasks created.')

[INFO] Tasks created.


In [61]:
print(tasks_data.shape)
print(tasks_data.dtype)
print(tasks_targets.shape)
print(tasks_targets.dtype)
print(len(tasks_info))
print(len(tasks_info[0]))

torch.Size([2, 100])
torch.float32
torch.Size([2, 100])
torch.float32
2
8


In [62]:
class InnerNetworkTask(Dataset):
    def __init__(self, data, targets, info):
        self.data = data 
        self.targets = targets
        self.info = info

    def __len__(self):
        assert len(self.data) == config['n_x'], '[ERROR] Length should be the same as n_x.'
        return len(self.data)

    def __getitem__(self, index):
        assert self.data[index].dtype == torch.float32, f'[ERROR] Expected type torch.float32, got type: {self.data[index].dtype}'
        assert self.targets[index].dtype == torch.float32, f'[ERROR] Expected type torch.float32, got type: {self.targets[index].dtype}'
        sample = {
            'x' : self.data[index],
            'y' : self.targets[index],
            'info' : self.info
        }
        return sample
    
    def __str__(self):
        return f'[INFO] InnerNetworkTask(data={self.data}, targets={self.targets}, info={self.info})'

In [63]:
# what the pool has become
initial_input_layer = torch.nn.Linear(in_features=config['in_features'], out_features=config['n_nodes_per_layer'])
initial_hidden_layer = torch.nn.Linear(in_features=config['n_nodes_per_layer'], out_features=config['n_nodes_per_layer'])
initial_output_layer = torch.nn.Linear(in_features=config['n_nodes_per_layer'], out_features=config['out_features'])
torch.nn.init.xavier_uniform_(initial_input_layer.weight)
torch.nn.init.xavier_uniform_(initial_hidden_layer.weight)
torch.nn.init.xavier_uniform_(initial_output_layer.weight)

Parameter containing:
tensor([[-0.1660, -0.3857,  0.2358, -0.9485]], requires_grad=True)

In [70]:
class InnerNetwork(gymnasium.Env, torch.nn.Module):
    def __init__(self, 
                task: InnerNetworkTask,
                epoch: int=0,
                in_features: int=config['in_features'],
                out_features: int=config['out_features'],
                learning_rate: float=config['learning_rate'],
                batch_size: int=config['batch_size'],
                action_cache_size: float=config['action_cache_size'],
                num_workers: int=config['num_workers'],
                shuffle: bool=True,
                ):
        super(InnerNetwork, self).__init__()
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.task = task
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.action_cache_size = action_cache_size
        self.num_workers = num_workers
        
        self.timestep = 0
        self.loss_vals = []
        self.reward_vals = []
        self.prev = defaultdict(lambda: None)
        self.curr = defaultdict(lambda: None)
        self.data_loader = DataLoader(task, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        self.data_iter = iter(self.data_loader)

        # TODO is check whether need to min max scale the reward 
        
        self.initial_input_layer = copy.deepcopy(initial_input_layer)
        self.initial_hidden_layer = copy.deepcopy(initial_hidden_layer)
        self.target_layer = copy.deepcopy(initial_hidden_layer)
        self.initial_output_layer = copy.deepcopy(initial_output_layer)
        self.layers = torch.nn.ModuleList([self.initial_input_layer, self.initial_hidden_layer, self.initial_output_layer]) 
        self.loss_fn = torch.nn.MSELoss()
        self.opt = torch.optim.Adam(self.layers.parameters(), lr=self.learning_rate)

        # create oracle (trained hidden layer) for reward signal
        self.train()
        for _ in range(1000):
            self.next_batch()
            self.opt.zero_grad()
            self.forward(self.curr['x'])
            loss = self.loss_fn(self.curr['y'], self.curr['y_hat'])
            loss.backward()
            self.opt.step()
        self.oracle_layer = self.layers[-2]

        # replace oracle with target 
        self.layers[-2] = self.target_layer

        # check initial loss with target
        self.next_batch()
        self.opt = torch.optim.Adam(self.layers.parameters(), lr=self.learning_rate)
        self.forward(self.curr['x'])
        self.curr['loss'] = self.loss_fn(self.curr['y'], self.curr['y_hat'])
        self.curr['loss'].backward()

        self.observation_space = gymnasium.spaces.box.Box(low=float('-inf'), high=float('inf'), shape=self.build_state().shape)
        # actions are now "steps" so they need to be smaller
        self.action_space = gymnasium.spaces.box.Box(low=-0.1, high=0.1, shape=(config['n_nodes_per_layer'] * config['n_nodes_per_layer'], ), dtype=np.float32)
        print(f"[INFO] Initialized target network for task")

    def step(self, action: np.int64) -> Tuple[torch.Tensor, float, bool, dict]: 
        self.timestep += 1

        # update target layer
        nudges = action
        print(f"weights before={self.target_layer.weight}")
        print(f"nudges={nudges}")
        weights = self.target_layer.weight.reshape((config['n_nodes_per_layer'] * config['n_nodes_per_layer'], ))
        new_weights = weights.detach().numpy() + nudges
        self.target_layer.weight.data = torch.tensor(new_weights).reshape((config['n_nodes_per_layer'], config['n_nodes_per_layer']))
        print(f"weights after={self.target_layer.weight}")
        print()

        # update gradient
        self.prev = self.curr
        self.curr = defaultdict(lambda: None)
        self.next_batch()
        self.train()
        self.opt.zero_grad()
        self.forward(self.curr['x'])
        self.curr['loss'] = self.loss_fn(self.curr['y'], self.curr['y_hat'])
        self.curr['loss'].backward()

        # get next state
        # get learning signal
        # log data
        s_prime = self.build_state() # layer, gradient, task info
        termination = torch.any(torch.abs(self.target_layer.weight.data) > 3)
        learning_signal = self.learning_signal()
        self.log()

        return (
            s_prime,
            learning_signal, 
            termination,
            False,
            {}
        )
            
    def next_batch(self, throw_exception=False) -> None:
        if (throw_exception):
            batch = next(self.data_iter)
            self.curr['x'] = batch['x'].view(-1, 1)
            self.curr['y'] = batch['y'].view(-1, 1)
            self.curr['info'] = batch['info']
        else: 
            try:
                batch = next(self.data_iter)
            except StopIteration:
                self.data_loader = DataLoader(self.task, batch_size=self.batch_size, shuffle=self.shuffle, num_workers=self.num_workers)
                self.data_iter = iter(self.data_loader)
                batch = next(self.data_iter)
            finally:
                self.curr['x'] = batch['x'].view(-1 ,1)
                self.curr['y'] = batch['y'].view(-1, 1)
                self.curr['info'] = batch['info']

    def forward(self, x) -> None:
        for i in range(len(self.layers) - 1): 
            x = torch.nn.functional.relu(self.layers[i](x))
        self.curr['y_hat'] = self.layers[-1](x) 
    
    def build_state(self) -> np.ndarray:
        # characterize the task
        task_info = torch.tensor([self.task.info['amp'], self.task.info['phase_shift']]).squeeze()
        loss = torch.Tensor([self.curr['loss']])
        yhat_scale = torch.Tensor([torch.Tensor(torch.max(torch.abs(self.curr['y_hat']))).detach().item()])
        # get gradient for layer being developed
        gradients = [layer.weight.grad for layer in self.layers]
        gradient = gradients[-2].reshape((config['n_nodes_per_layer'] * config['n_nodes_per_layer'], ))
        # get params of layer being developed
        params = self.layers[-2].weight.reshape((config['n_nodes_per_layer'] * config['n_nodes_per_layer'],))
        
        return torch.concat((
            # target 
            params,

            # helpful info
            # task_info,
            # loss,
            # yhat_scale,
            # gradient,
        ), dim=0).detach().numpy()
    
    def learning_signal(self) -> torch.Tensor: 
        # TODO: try target network's loss
        # TODO: try difference between deltas rather than difference between layers
        # signal = - self.curr['loss']

        loss_fn = torch.nn.MSELoss()
        signal = - loss_fn(self.target_layer.weight.data, self.oracle_layer.weight.data)
        high_magnitude = torch.any(torch.abs(self.target_layer.weight.data) > 3)
        if high_magnitude:
            signal -= 100
        self.curr['reward'] = signal
        
        print(f"[INFO] signal={signal}")
        return signal
        # TODO: try combined learning signal as (difference from oracle) + (loss)

    def log(self):
        task_num = str(self.task.info['i'])
        self.loss_vals.append(copy.copy(self.curr['loss']))
        self.reward_vals.append(copy.copy(self.curr['reward']))
        wandb.log({ f'loss_task{task_num}_per_step' : self.curr['loss']})
        wandb.log({ f'reward_task{task_num}_per_step' : self.curr['reward']})

    def reset(self, seed=None) -> np.ndarray:
        print(f'[INFO] Reset at {self.timestep}')
        self.initial_input_layer = copy.deepcopy(initial_input_layer)
        self.initial_hidden_layer = copy.deepcopy(initial_hidden_layer)
        self.target_layer = copy.deepcopy(initial_hidden_layer)
        self.initial_output_layer = copy.deepcopy(initial_output_layer)
        self.layers = torch.nn.ModuleList([self.initial_input_layer, self.initial_hidden_layer, self.initial_output_layer]) 
        self.loss_fn = torch.nn.MSELoss()
        self.opt = torch.optim.Adam(self.layers.parameters(), lr=self.learning_rate)

        # create oracle (trained hidden layer) for reward signal
        self.train()
        for _ in range(1000):
            self.next_batch()
            self.opt.zero_grad()
            self.forward(self.curr['x'])
            loss = self.loss_fn(self.curr['y'], self.curr['y_hat'])
            loss.backward()
            self.opt.step()
        self.oracle_layer = self.layers[-2]

        # replace oracle with target 
        self.layers[-2] = self.target_layer

        # check initial loss with target
        self.next_batch()
        self.opt = torch.optim.Adam(self.layers.parameters(), lr=self.learning_rate)
        self.forward(self.curr['x'])
        self.curr['loss'] = self.loss_fn(self.curr['y'], self.curr['y_hat'])
        self.curr['loss'].backward()
        return self.build_state(), None

In [71]:
class REML:
    def __init__(
        self,
        tasks: List[InnerNetworkTask],
        model=config['sb3_model'],
        policy=config['sb3_policy'],
        epochs: int=config['epochs'],
        timesteps: int=config['timesteps'],
        device: str=config['device'],
        log_dir: str=f"./{config['log_dir']}/{config['sb3_model']}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        ):
        self.tasks = tasks
        if config['sb3_model']=='PPO':
            model = PPO
        elif config['sb3_model']=='RecurrentPPO':
            model = RecurrentPPO
        elif config['sb3_model']=='A2C':
            model = A2C
        dummy_env = self.make_env(tasks[0])
        self.model = SAC(policy, dummy_env, tensorboard_log=log_dir)
        self.policy = policy
        self.epochs = epochs
        self.timesteps = timesteps
        self.device = device # TODO is to check whether cuda is used as assumed
        self.log_dir = log_dir

    
    def __str__(self) -> str:
        return f'REML(model={self.model}, policy={self.policy})'
    
    def make_env(self, task, epoch=None) -> gymnasium.Env:
        return gymnasium.wrappers.NormalizeObservation(InnerNetwork(task, epoch=epoch))

    def train(self):
        # wraps stablebaselines learn() so we call it n * m times
        # n is the number of epochs where we run all m tasks
        # we use the same policy, swapping out envs for the n tasks, m times. 

        # to calculate variance
        # e.g., task: [ n: [epoch: [100 values]] ] / array with n rows, epoch columns 
        # where cell @ [nth run][mth epoch] is cumulative loss/reward
        return_taskkey_epochcol = defaultdict(lambda: [])
        cumuloss_taskkey_epochcol = defaultdict(lambda: [])
        for epoch in range(self.epochs):
            print(f'[INFO] Epoch={epoch + 1}/{self.epochs}')
            for i, task in enumerate(self.tasks): 
                self.task = task
                print(f'[INFO] Task={i+1}/{len(self.tasks)}')

                # each task gets its own network
                self.env = self.make_env(self.task, epoch=epoch)
                self.model.set_env(self.env)
                self.model.learn(total_timesteps=self.timesteps)

                # track reward and loss for plots
                return_taskkey_epochcol[str(self.task.info['i'])].append(sum(self.env.reward_vals))
                cumuloss_taskkey_epochcol[str(self.task.info['i'])].append(sum(self.env.loss_vals))

                # log to wandb
                wandb.log({ f'cumulative_reward_task{i}_per_epoch' : sum(self.env.reward_vals) })
                wandb.log({ f'cumulative_loss_task{i}_per_epoch' : sum(self.env.loss_vals) })

                # sine curves
                self.generate_sine_curve(epoch=epoch, task=i, image=True, title='training_sine_curves', args={'label' : f'task_{i}'})
                
        return return_taskkey_epochcol, cumuloss_taskkey_epochcol
    
    # def evaluate_convergence_speed(self, steps=100) -> dict:
    #     # generates loss curve over 'steps' per task
    #     # TODO is to add option to do so with std if n_runs>1

    #     lossperstep_bytask = defaultdict(lambda: [])

    #     for task in self.tasks: 
    #         env = self.make_env(task)
    #         self.model.set_env(env, force_reset=False)
    #         obs, _ = env.reset()

    #         while len(env.layers) < config['n_layers_per_network']:
    #             action, _ = self.model.predict(obs)
    #             obs, _, _, _, _ = env.step(action)

    #         for _ in range(steps):
    #             action, _ = self.model.predict(obs)
    #             obs, _, _, _, _ = env.step(action)
    #             lossperstep_bytask[task].append(env.curr['loss'])

    #     return lossperstep_bytask

    def generate_sine_curve(self, env=None, data=None, epoch=None, task=None, image=False, new_figures=False, title=None, args=defaultdict()) -> List:
        # generates sine curve after 'env.layers' is full, with option to set env, limit to 
        # subset of env data (for few shot evaluation), and to create png

        # if env is not None:
        #     self.env = env
        #     self.model.set_env(env, force_reset=False)

        # self.env.eval()
        # obs, _ = self.env.reset()
        
        # while len(self.env.layers)!=config['n_layers_per_network']:
        #     action, _ = self.model.predict(obs)
        #     obs, _, _, _, _ = self.env.step(action)
        
        # if data is specified, wrap in new task
        # if data is not specified, the iterator is used over set
        if data is not None:
            dataset = InnerNetworkTask(data=data[:, 0].clone(), targets=data[:, 1].clone(), info=self.task.info)
        else: 
            dataset = self.task

        xs, ys = dataset.data.clone(), dataset.targets.clone()
        xs, ys = xs.view(len(xs), 1), ys.view(len(ys), 1)
        for i in range(len(self.env.layers) - 1): 
            xs = torch.nn.functional.relu(self.env.layers[i](xs))
        yhats = self.env.layers[-1](xs) 

        if new_figures:
            plt.figure()
        plot_title = title if title!=None else f'sine_curve_epoch_{epoch}_task_{task}' if epoch!=None and task!=None else 'sine_curve'
        plot_path = f'{self.log_dir}/{plot_title}.png'  
        plt.plot(dataset.data, [yhat.detach().numpy() for yhat in yhats], **args)
        plt.plot(dataset.data, dataset.targets, label='ground truth', linestyle='--')
        plt.title(plot_title)
        plt.legend()

        if image:
            plt.savefig(plot_path)
            wandb.log({plot_title: wandb.Image(plot_path)})
       
        xs, yhats = dataset.data, [yhat.detach().numpy() for yhat in yhats]
        return xs, yhats

### testing

In [72]:
tasks = [InnerNetworkTask(data=tasks_data[i], targets=tasks_targets[i], info=tasks_info[i]) for i in range(config['n_tasks'])]
eval_task = random.choice(list(tasks))
training_tasks = list(set(tasks) - {eval_task})
reml = REML(tasks=training_tasks)

[INFO] Initialized target network for task


In [73]:
path = f"meta_{config['sb3_model']}_{datetime.datetime.now().strftime('%H-%M')}"
return_taskkey_nrow_epochcol = defaultdict(lambda: [])
cumuloss_taskkey_nrow_epochcol = defaultdict(lambda: [])

#        epoch1  epoch2  epoch3 ... <- epochs as cols
# task: [return, return, return, ...] 
# task: [cumulative loss, cumulative loss, cumulative loss, ...] 
for n in range(config['n_runs']): 
    print(f"[INFO] n={n+1}")
    return_taskkey_epochvalues, cumuloss_taskkey_epochvalues = reml.train()
    for task in tasks:
        return_taskkey_nrow_epochcol[str(task.info['i'])].append(return_taskkey_epochvalues[str(task.info['i'])])
        cumuloss_taskkey_nrow_epochcol[str(task.info['i'])].append(cumuloss_taskkey_epochvalues[str(task.info['i'])])

# save to json
with open(f'returns_{path}', 'w') as json_file:
    json.dump(return_taskkey_nrow_epochcol, json_file, indent=4)
with open(f'cumuloss_{path}', 'w') as json_file:
    json.dump(cumuloss_taskkey_nrow_epochcol, json_file, indent=4)

[INFO] n=1
[INFO] Epoch=1/3
[INFO] Task=1/1
[INFO] Initialized target network for task
[INFO] Reset at 0
weights before=Parameter containing:
tensor([[-0.7951,  0.6267, -0.8307, -0.7619],
        [-0.3047,  0.4708, -0.1719,  0.8580],
        [-0.1162,  0.4736, -0.5573, -0.4005],
        [ 0.2186,  0.2895, -0.1463, -0.2233]], requires_grad=True)
nudges=[ 0.02056923  0.07541253 -0.0338899  -0.02612297  0.00858621 -0.09026112
  0.09772592  0.09210934  0.08166606  0.06629644 -0.05115603  0.04367546
  0.05430412 -0.04125778 -0.06207305  0.0731343 ]
weights after=Parameter containing:
tensor([[-0.7746,  0.7021, -0.8646, -0.7880],
        [-0.2961,  0.3805, -0.0742,  0.9501],
        [-0.0345,  0.5399, -0.6085, -0.3568],
        [ 0.2729,  0.2483, -0.2083, -0.1502]], requires_grad=True)

[INFO] signal=-0.944596529006958
weights before=Parameter containing:
tensor([[-0.7746,  0.7021, -0.8646, -0.7880],
        [-0.2961,  0.3805, -0.0742,  0.9501],
        [-0.0345,  0.5399, -0.6085, -0.3568],


### meta analysis

#### speed of training test

In [ ]:
# get loss for meta trained network per step
reml = REML(layer_pool=pool, tasks=tasks)
reml.model.load(path)
meta_lossperstep_bytask = reml.evaluate_loss_curves()

# get loss for vanilla network per step
vanilla_lossperstep_bytask = {}
for task, x, y in zip(tasks, tasks_data, tasks_targets):
    model = RegressionModel()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    vanilla_lossperstep_bytask[task] = []
    for epoch in range(100):
        optimizer.zero_grad() 
        outputs = model(x.view(-1, 1))
        loss = criterion(outputs, y.view(-1,1))
        loss.backward()
        optimizer.step()
        vanilla_lossperstep_bytask[task].append(loss)


In [ ]:
for i, (x, y) in enumerate(zip(tasks_data, tasks_targets)):
    task = tasks[i]
    vanilla_label = f'Vanilla network: Task {i+1}' 
    meta_label = f'Meta-trained network: Task {i+1}' 
    meta_loss_vals = [val.detach().numpy() for val in meta_lossperstep_bytask[task]]
    vanilla_loss_vals = [val.detach().numpy() for val in vanilla_lossperstep_bytask[task]]
    plt.plot(range(100), vanilla_loss_vals, label=vanilla_label)
    plt.plot(range(100), meta_loss_vals, label=meta_label)
    plt.xlabel('Timestep')
    plt.ylabel('MSE loss')
    plt.legend()
    plt.show()


#### few shot learning tests

k=5

In [ ]:
# unseen task
# choose 5 and 10 datapoints on it, K = 5, K = 10
k = 5
k_pairs = torch.tensor(random.sample(list(zip(eval_task.data, eval_task.targets)), k))
print(len(k_pairs))
print(k_pairs)

# network still recieves the same 100 x values {-5, ..., 5}
# the difference is that a target value is only provided for 5 or 10 of these 100 values
# start by generating the 'pre-update' curve 
# next pass k datapoints and do 1 gradient update step to get the curve
# next do 9 more gradient update steps (total 10) to get next curve
# plot 'pre-update', '1 grad step', '10 grad steps' curves together with the 'k_pairs' data points

In [ ]:
# env with eval task
reml = REML(layer_pool=pool, tasks=tasks)
reml.model.load(path)
env = reml.make_env(eval_task, pool)
reml.model.set_env(env)
reml.env = env
reml.task = eval_task

# ground truth
plt.plot(eval_task.data, eval_task.targets, linestyle='--', label='ground truth')

# pre-update
reml.generate_sine_curve(env=env, args={'linestyle' : '--', 'label': 'pre-update'})

# k points 
plt.scatter(k_pairs[:, 0], k_pairs[:, 1], marker='^', color='b', label='k points')

# train for 1 grad step
reml.env.batch_size = k
obs, _ = reml.env.reset()
for _ in range(1):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' :'1 grad step'})
    
# train for 10 grad steps
obs, _ = reml.env.reset()
for _ in range(10):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' : '10 grad steps'})
plt.title('REML, k=5')

k=10

In [ ]:
k = 10
k_pairs = torch.tensor(random.sample(list(zip(eval_task.data, eval_task.targets)), k))
print(len(k_pairs))
print(k_pairs)

In [ ]:
reml = REML(layer_pool=pool, tasks=tasks)
reml.model.load(path)
env = reml.make_env(eval_task)
reml.model.set_env(env)
reml.env = env
reml.task = eval_task

# ground truth
plt.plot(eval_task.data, eval_task.targets, linestyle='--', label='ground truth')

# pre-update
reml.generate_sine_curve(env=env, args={'linestyle' : '--', 'label': 'pre-update'})

# k points 
plt.scatter(k_pairs[:, 0], k_pairs[:, 1], marker='^', color='b', label='k points')

# train for 1 grad step
reml.env.batch_size = k
obs, _ = reml.env.reset()
for _ in range(1):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' :'1 grad step'})
    
# train for 10 grad steps
obs, _ = reml.env.reset()
for _ in range(10):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' : '10 grad steps'})
plt.title('REML, k=10')

## episodic learning 

In [ ]:
config['episodic'] = True
config

In [ ]:
tasks = [InnerNetworkTask(data=tasks_data[i], targets=tasks_targets[i], info=tasks_info[i]) for i in range(config['n_tasks'])]
eval_task = random.choice(list(tasks))
training_tasks = list(set(tasks) - {eval_task})
pool = LayerPool(layers=layers) if config['pretrain'] else LayerPool(layers=None)
reml = REML(layer_pool=pool, tasks=training_tasks)

In [ ]:
reml.train()
path = f"meta_episodic_{config['sb3_model']}_{datetime.datetime.now().strftime('%H-%M')}"
reml.model.save(path)

In [ ]:
# get loss for meta trained network per step
reml = REML(layer_pool=pool, tasks=tasks)
reml.model.load(path)
meta_lossperstep_bytask = reml.evaluate_loss_curves()

# get loss for vanilla network per step
vanilla_lossperstep_bytask = {}
for task, x, y in zip(tasks, tasks_data, tasks_targets):
    model = RegressionModel()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    model.train()
    vanilla_lossperstep_bytask[task] = []
    for epoch in range(100):
        optimizer.zero_grad() 
        outputs = model(x.view(-1, 1))
        loss = criterion(outputs, y.view(-1,1))
        loss.backward()
        optimizer.step()
        vanilla_lossperstep_bytask[task].append(loss)


In [ ]:
for i, (x, y) in enumerate(zip(tasks_data, tasks_targets)):
    task = tasks[i]
    vanilla_label = f'Vanilla network: Task {i+1}' 
    meta_label = f'Meta-trained network: Task {i+1}' 
    meta_loss_vals = [val.detach().numpy() for val in meta_lossperstep_bytask[task]]
    vanilla_loss_vals = [val.detach().numpy() for val in vanilla_lossperstep_bytask[task]]
    plt.plot(range(100), vanilla_loss_vals, label=vanilla_label)
    plt.plot(range(100), meta_loss_vals, label=meta_label)
    plt.xlabel('Timestep')
    plt.ylabel('MSE loss')
    plt.legend()
    plt.show()


#### k=5

In [ ]:
k = 5
k_pairs = torch.tensor(random.sample(list(zip(eval_task.data, eval_task.targets)), k))
print(len(k_pairs))
print(k_pairs)

In [ ]:
# env with eval task
reml = REML(layer_pool=pool, tasks=tasks)
reml.model.load(path)
env = reml.make_env(eval_task, pool)
reml.model.set_env(env)
reml.env = env
reml.task = eval_task

# ground truth
plt.plot(eval_task.data, eval_task.targets, linestyle='--', label='ground truth')

# pre-update
reml.generate_sine_curve(env=env, args={'linestyle' : '--', 'label': 'pre-update'})

# k points 
plt.scatter(k_pairs[:, 0], k_pairs[:, 1], marker='^', color='b', label='k points')

# train for 1 grad step
reml.env.batch_size = k
obs, _ = reml.env.reset()
for _ in range(1):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' :'1 grad step'})
    
# train for 10 grad steps
obs, _ = reml.env.reset()
for _ in range(10):
    action, _ = reml.model.predict(obs)
    obs, _, _, _, _ = reml.env.step(action)
reml.generate_sine_curve(env=env, args={'label' : '10 grad steps'})
plt.title('REML, k=5')